## Dependencies / Setup

In [1]:
!pip install census

distributed 1.21.8 requires msgpack, which is not installed.


In [16]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
import datetime
from census import Census

# Today's Date
now = datetime.datetime.now()

# PyMySQL
import pymysql
pymysql.install_as_MySQLdb()

# Import API key
from config import api_key
c = Census(api_key, year=2017)

#Today's date + HourMinute
timetag = now.strftime("%Y_%m_%d_%H%M")
print(timetag)

2019_02_02_1114


## Perform API Call

In [17]:
# Run Census Search to retrieve data on all zip codes (2017 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME",
                          "B19013_001E", 
                          "B01003_001E", 
                          "B01002_001E",
                          "B19301_001E",
                          "B23025_005E",
                          "B17001_002E"), {'for':'county:*', 'in': 'state:*'})
#Convert to a Dataframe
census_pd = pd.DataFrame(census_data)
census_pd.head()

,B01002_001E,B01003_001E,B17001_002E,B19013_001E,B19301_001E,B23025_005E,NAME,county,state
0,38.1,34933.0,19146.0,14752.0,7887.0,3296.0,"Corozal Municipio, Puerto Rico",047,72
1,43.5,11297.0,5945.0,17636.0,8830.0,735.0,"Maunabo Municipio, Puerto Rico",095,72
2,36.4,21661.0,12379.0,16868.0,7983.0,1601.0,"Peñuelas Municipio, Puerto Rico",111,72
3,39.5,148863.0,75187.0,16561.0,10775.0,7013.0,"Ponce Municipio, Puerto Rico",113,72
4,42.0,38970.0,21472.0,14275.0,8072.0,3307.0,"San Sebastián Municipio, Puerto Rico",131,72


In [18]:
census_pd['geoid'] = census_pd['state'] +  census_pd['county']
census_pd.head()

,B01002_001E,B01003_001E,B17001_002E,B19013_001E,B19301_001E,B23025_005E,NAME,county,state,geoid
0,38.1,34933.0,19146.0,14752.0,7887.0,3296.0,"Corozal Municipio, Puerto Rico",047,72,72047
1,43.5,11297.0,5945.0,17636.0,8830.0,735.0,"Maunabo Municipio, Puerto Rico",095,72,72095
2,36.4,21661.0,12379.0,16868.0,7983.0,1601.0,"Peñuelas Municipio, Puerto Rico",111,72,72111
3,39.5,148863.0,75187.0,16561.0,10775.0,7013.0,"Ponce Municipio, Puerto Rico",113,72,72113
4,42.0,38970.0,21472.0,14275.0,8072.0,3307.0,"San Sebastián Municipio, Puerto Rico",131,72,72131


In [19]:
#Column REordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", 
                                      "county": "County",
                                      "geoid": "geoid"})

#Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Name", "County","geoid", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Unemployment Count", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

3220


,Name,County,geoid,Population,Median Age,Household Income,Per Capita Income,Unemployment Count,Poverty Count,Poverty Rate
0,"Corozal Municipio, Puerto Rico",047,72047,34933.0,38.1,14752.0,7887.0,3296.0,19146.0,54.807775
1,"Maunabo Municipio, Puerto Rico",095,72095,11297.0,43.5,17636.0,8830.0,735.0,5945.0,52.624591
2,"Peñuelas Municipio, Puerto Rico",111,72111,21661.0,36.4,16868.0,7983.0,1601.0,12379.0,57.148793
3,"Ponce Municipio, Puerto Rico",113,72113,148863.0,39.5,16561.0,10775.0,7013.0,75187.0,50.507514
4,"San Sebastián Municipio, Puerto Rico",131,72131,38970.0,42.0,14275.0,8072.0,3307.0,21472.0,55.098794


In [20]:
census_pd.describe()

,Population,Median Age,Household Income,Per Capita Income,Unemployment Count,Poverty Count,Poverty Rate
count,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3.220000e+03,3220.000000
mean,1.007681e+05,41.123882,48994.966770,25657.033230,3348.106211,1.465639e+04,16.174182
std,3.244996e+05,5.325031,13877.178398,6667.520452,12264.417369,5.116298e+04,8.078167
min,7.400000e+01,21.600000,11680.000000,5943.000000,0.000000,1.000000e+01,2.424187
25%,1.121350e+04,38.000000,40622.000000,21568.000000,295.000000,1.664500e+03,11.055315
50%,2.584750e+04,41.100000,47636.500000,25139.000000,781.500000,4.240500e+03,14.808937
75%,6.660825e+04,44.200000,55476.000000,28997.000000,2174.500000,1.079175e+04,19.108649
max,1.010572e+07,66.400000,129588.000000,69529.000000,406426.000000,1.688505e+06,65.146881


In [21]:
#Split the county and states into separate columns 
census_pd[['County','State']] = census_pd.Name.str.split(",",expand=True)
#Strip excess spaces in the State column 
census_pd['State'] = census_pd['State'].str.strip()

In [22]:
#import the state abbreviation table
stateAbbr = pd.read_csv("states.csv")
stateAbbr.head()

,State,abbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [23]:
##merge the state abbreviation table with the census table
census_pd = pd.merge(census_pd,stateAbbr,how = 'left',on = ['State'])
census_pd.head()

,Name,County,geoid,Population,Median Age,Household Income,Per Capita Income,Unemployment Count,Poverty Count,Poverty Rate,State,abbr
0,"Corozal Municipio, Puerto Rico",Corozal Municipio,72047,34933.0,38.1,14752.0,7887.0,3296.0,19146.0,54.807775,Puerto Rico,PR
1,"Maunabo Municipio, Puerto Rico",Maunabo Municipio,72095,11297.0,43.5,17636.0,8830.0,735.0,5945.0,52.624591,Puerto Rico,PR
2,"Peñuelas Municipio, Puerto Rico",Peñuelas Municipio,72111,21661.0,36.4,16868.0,7983.0,1601.0,12379.0,57.148793,Puerto Rico,PR
3,"Ponce Municipio, Puerto Rico",Ponce Municipio,72113,148863.0,39.5,16561.0,10775.0,7013.0,75187.0,50.507514,Puerto Rico,PR
4,"San Sebastián Municipio, Puerto Rico",San Sebastián Municipio,72131,38970.0,42.0,14275.0,8072.0,3307.0,21472.0,55.098794,Puerto Rico,PR


In [24]:
#calculate and add column for unemployement rate
census_pd['Unemployment Rate'] = census_pd['Unemployment Count']/census_pd['Population']
census_pd.head()

,Name,County,geoid,Population,Median Age,Household Income,Per Capita Income,Unemployment Count,Poverty Count,Poverty Rate,State,abbr,Unemployment Rate
0,"Corozal Municipio, Puerto Rico",Corozal Municipio,72047,34933.0,38.1,14752.0,7887.0,3296.0,19146.0,54.807775,Puerto Rico,PR,0.094352
1,"Maunabo Municipio, Puerto Rico",Maunabo Municipio,72095,11297.0,43.5,17636.0,8830.0,735.0,5945.0,52.624591,Puerto Rico,PR,0.065062
2,"Peñuelas Municipio, Puerto Rico",Peñuelas Municipio,72111,21661.0,36.4,16868.0,7983.0,1601.0,12379.0,57.148793,Puerto Rico,PR,0.073912
3,"Ponce Municipio, Puerto Rico",Ponce Municipio,72113,148863.0,39.5,16561.0,10775.0,7013.0,75187.0,50.507514,Puerto Rico,PR,0.047110
4,"San Sebastián Municipio, Puerto Rico",San Sebastián Municipio,72131,38970.0,42.0,14275.0,8072.0,3307.0,21472.0,55.098794,Puerto Rico,PR,0.084860


In [25]:
# Save the updated dataframe as a csv
census_pd.to_csv("county_census_data.csv", encoding="utf-8", index=False)
census_pd.head()

,Name,County,geoid,Population,Median Age,Household Income,Per Capita Income,Unemployment Count,Poverty Count,Poverty Rate,State,abbr,Unemployment Rate
0,"Corozal Municipio, Puerto Rico",Corozal Municipio,72047,34933.0,38.1,14752.0,7887.0,3296.0,19146.0,54.807775,Puerto Rico,PR,0.094352
1,"Maunabo Municipio, Puerto Rico",Maunabo Municipio,72095,11297.0,43.5,17636.0,8830.0,735.0,5945.0,52.624591,Puerto Rico,PR,0.065062
2,"Peñuelas Municipio, Puerto Rico",Peñuelas Municipio,72111,21661.0,36.4,16868.0,7983.0,1601.0,12379.0,57.148793,Puerto Rico,PR,0.073912
3,"Ponce Municipio, Puerto Rico",Ponce Municipio,72113,148863.0,39.5,16561.0,10775.0,7013.0,75187.0,50.507514,Puerto Rico,PR,0.047110
4,"San Sebastián Municipio, Puerto Rico",San Sebastián Municipio,72131,38970.0,42.0,14275.0,8072.0,3307.0,21472.0,55.098794,Puerto Rico,PR,0.084860
